In [1]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import types
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName("belgian_housing_buy_lease") \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/13 19:20:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql import SparkSession
from prefect_gcp import GcpCredentials
import pandas as pd
import pandas_gbq as pdbq
gcp_credentials_block = await GcpCredentials.load("belgium-housing-gcp-cred")
#Create PySpark SparkSession
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()



/home/daniel_gheorghita/anaconda3/envs/dezoomcamp/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2083: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(
/home/daniel_gheorghita/anaconda3/envs/dezoomcamp/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


root
 |-- Unnamed: 0: struct (nullable = true)
 |-- NISCode: long (nullable = true)
 |-- NameFre: string (nullable = true)
 |-- NameDut: string (nullable = true)
 |-- NameGer: string (nullable = true)
 |-- RegistrationType: string (nullable = true)
 |-- LessorType: string (nullable = true)
 |-- TakerType: string (nullable = true)
 |-- RentsNumber: double (nullable = true)
 |-- RentP25: double (nullable = true)
 |-- RentP50: double (nullable = true)
 |-- RentP75: double (nullable = true)
 |-- ChargesP25: double (nullable = true)
 |-- ChargesP50: double (nullable = true)
 |-- ChargesP75: double (nullable = true)
 |-- TotalRentP25: double (nullable = true)
 |-- TotalRentP50: double (nullable = true)
 |-- TotalRentP75: double (nullable = true)
 |-- Date: timestamp (nullable = true)



+----------+-------+---------------+------------+------------------+--------------------+-------------+-------------+-----------+------------+------------+--------+----------+------------+------------+------------+------------+------------+-------------------+
|Unnamed: 0|NISCode|        NameFre|     NameDut|           NameGer|    RegistrationType|   LessorType|    TakerType|RentsNumber|     RentP25|     RentP50| RentP75|ChargesP25|  ChargesP50|  ChargesP75|TotalRentP25|TotalRentP50|TotalRentP75|               Date|
+----------+-------+---------------+------------+------------------+--------------------+-------------+-------------+-----------+------------+------------+--------+----------+------------+------------+------------+------------+------------+-------------------+
|        {}|   3000|Région wallonne|Waals Gewest|Wallonische Region| GeneralRegistration|  LegalPerson|  LegalPerson|      768.0|       700.0|      1750.0|  4000.0|       0.0|62.666666667|       295.0|       750.0|   

In [54]:
files = [    'ArrondissementWideRealEstate',
             'MunicipalityWideRealEstate',
             'NationalWideRealEstate',
             'ProvincialWideRealEstate',
             'RegionalWideRealEstate',
             'StatisticalUnitWideRealEstateTransactions',
             'DivisionWideRealEstateTransactions',
             ]

for file in [files[1]]:
    print(file)
    leases_df = pdbq.read_gbq(query_or_table=f'belgium_housing_leases.{file}Rents_all',
                       project_id='belgium-housing-market',
                      credentials=gcp_credentials_block.get_credentials_from_service_account())
    leases_df = leases_df.drop_duplicates()
    #print(leases_df.describe())
    #print(leases_df.head())
    #print(leases_df['NISCode'].value_counts())
    #file = 'DivisionWideRealEstate'
    transactions_df = pdbq.read_gbq(query_or_table=f'belgium_housing_transactions.{file}Transactions_all',
                       project_id='belgium-housing-market',
                      credentials=gcp_credentials_block.get_credentials_from_service_account())
    transactions_df = transactions_df.drop_duplicates()
    #print(transactions_df.describe())
    #print(transactions_df.head())
    #print(transactions_df['NISCode'].value_counts())

    #Create PySpark DataFrame from Pandas
    leases_psdf = spark.createDataFrame(leases_df) 
    transactions_psdf = spark.createDataFrame(transactions_df)
    
    # Print schemas
    #leases_psdf.printSchema()
    #transactions_psdf.printSchema()
    
    # Select certain columns
    leases_psdf = leases_psdf.select("NISCode", "NameFre", "NameDut", "NameGer", "RentP50", "Date", "RentsNumber")
    transactions_psdf = transactions_psdf.select("NISCode", "NameFre", "NameDut", "NameGer", "PriceP50", "Date", "ParcelsNumber")
    
    # Print schemas
    #leases_psdf.printSchema()
    #transactions_psdf.printSchema()
    
    # Means
    leases_psdf = \
    leases_psdf \
        .groupBy("NISCode", "Date") \
        .agg(F.avg("RentP50").alias("RentP50"), \
             F.sum("RentsNumber").alias("RentsNumber"),\
             F.first("NameFre").alias("NameFre")) \
        #.filter(F.col("NISCode")==71002) \
        #.sort("Date") \
        #.show(truncate=False)
    
    transactions_psdf = \
        transactions_psdf \
            .groupBy("NISCode", "Date") \
            .agg(F.avg("PriceP50").alias("PriceP50"), \
                 F.sum("ParcelsNumber").alias("ParcelsNumber"),\
                 F.first("NameDut").alias("NameDut"),\
                 F.first("NameGer").alias("NameGer")) \
        #.filter(F.col("NISCode")==71002) \
        #.sort("Date") \
        #.show(truncate=False)

    # PySpark join multiple columns
    transactions_and_leases_psdf = \
        leases_psdf \
            .join(transactions_psdf, ["NISCode","Date"],"inner") \
            .sort("NISCode", "Date") \
            #.show()
    
    # Add column ratio price/rent
    transactions_and_leases_psdf = transactions_and_leases_psdf.withColumn('PriceToRentRatio',
                       transactions_and_leases_psdf["PriceP50"] / transactions_and_leases_psdf["RentP50"])
    transactions_and_leases_psdf.show()
    
    #sparkDF.show()
    
    # PySpark to Pandas DF
    transactions_and_leases_df = transactions_and_leases_psdf.toPandas()
    
    # Upload Pandas df to BQ
    transactions_and_leases_df.to_gbq(
            destination_table=f"belgium_housing_transactions_rents.{file}",
            project_id="belgium-housing-market",
            credentials=gcp_credentials_block.get_credentials_from_service_account(),
            chunksize=500000,
            if_exists="replace")
    

MunicipalityWideRealEstate


/home/daniel_gheorghita/anaconda3/envs/dezoomcamp/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2083: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(
/home/daniel_gheorghita/anaconda3/envs/dezoomcamp/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2083: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(
/home/daniel_gheorghita/anaconda3/envs/dezoomcamp/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


23/04/14 22:06:47 WARN TaskSetManager: Stage 109 contains a task of very large size (1164 KiB). The maximum recommended task size is 1000 KiB.
+-------+-------------------+------------------+-----------+----------+--------+-------------+----------+----------+------------------+
|NISCode|               Date|           RentP50|RentsNumber|   NameFre|PriceP50|ParcelsNumber|   NameDut|   NameGer|  PriceToRentRatio|
+-------+-------------------+------------------+-----------+----------+--------+-------------+----------+----------+------------------+
|  11001|2016-03-31 00:00:00|          1385.375|       60.0|Aartselaar|300000.0|         26.0|Aartselaar|Aartselaar|216.54786610123614|
|  11001|2016-06-30 00:00:00| 658.3333333333334|       35.0|Aartselaar|337000.0|         26.0|Aartselaar|Aartselaar|511.89873417721515|
|  11001|2016-09-30 00:00:00|            2242.5|       66.0|Aartselaar|305000.0|         31.0|Aartselaar|Aartselaar|136.00891861761426|
|  11001|2016-12-31 00:00:00| 881.6666666

/home/daniel_gheorghita/anaconda3/envs/dezoomcamp/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:194: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
